In [29]:
#!python3 --version
#from pyspark import SparkContext
#sc = SparkContext.getOrCreate()
#print(sc.pythonVer)


In [30]:
#!pip list


In [1]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\paula\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [32]:
#!pip uninstall -y pyspark
#pip uninstall -yspark

In [33]:
#!pip cache purge

#!pip install pyspark==3.5.2


In [34]:
#Libreria para análisis EDA
#!pip install matplotlib seaborn sweetviz


In [2]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\paula\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\paula\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
#Importar librerias
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import glob
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import os
import sys

In [5]:

file_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\LOCardReports\Cash_Advance_Transaction-20240903-0921.xls'

# Especificar la ruta del archivo Excel
output_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosSalida\Concatenado2.xlsx'

# Especificar la ruta del archivo Excel
output_path3 = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosSalida\Concatenado3.xlsx'

# Ruta de la carpeta que contiene los archivos de Entrada
folder_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\LOCardReports'
folder_path_tr_bb = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\LO_TR_BBReports'
host_folder_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\HostReports'
Logs_path = r'C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\Logs'


In [ ]:
#------------------------------------------------------------------SPARK Consolidado

#import os
#os.environ["PYSPARK_PYTHON"] = "python3"  # O "python" si ese es tu ejecutable
spark = SparkSession.builder.appName("Host_LO Comparative").getOrCreate()

#Pandas a Spark

dfLO = spark.createDataFrame(df_combined)
dfHOST = spark.createDataFrame(df_host)


#Renombre de columnas
dfLO = dfLO.withColumnRenamed("HOSTSEQ NUMBER", "hostseq_number")
dfHOST = dfHOST.withColumnRenamed("Seq", "seq")

# Filtrar el DataFrame dfLO
#dfLO = dfLO.filter(F.col("STATUS") != "Denied")
dfLO = dfLO.filter(F.col("TRANSACTIONTYPE") != "Balance inquiry")


#Verificación
#print('Dataframe de LiveOffice')
#dfLO.printSchema()
#print('Dataframe de Host')
#dfHOST.printSchema()

joined_df = dfLO.join(
    dfHOST,
    dfLO["hostseq_number"] == dfHOST["seq"],
    "left"
)


# Agregar columna de indicador
result_df = joined_df.withColumn(
    "found_in_datastream",
    F.when(F.col("seq").isNotNull(), True).otherwise(False)
)

result_df.show()

#unmatched_df.show()
# Convertir el DataFrame de Spark a Pandas
pandas_df = result_df.toPandas()
pandas_df.to_excel(output_path3, index=False, engine='openpyxl')  # Usa openpyxl como motor
print(f"Archivo exportado exitosamente a {output_path3}")

In [41]:
#-------------------------------------------------------Logs Extraction-------------------------------------------------------------------------------
# Crear una lista para almacenar los DataFrames
logs_dataframes = []
logs_dataframes_tr = []
logs_dataframes_bb = []


column_names = ["FilteredData"]
df_LogsUnified=  pd.DataFrame()



for subcarpeta in os.listdir(Logs_path):
    ruta_subcarpeta = os.path.join(Logs_path, subcarpeta)
    # Verificar si es una carpeta
    if os.path.isdir(ruta_subcarpeta) and os.path.basename(ruta_subcarpeta).lower() == 'atm':
        print(ruta_subcarpeta)
        for archivo in os.listdir(ruta_subcarpeta):
            if archivo.endswith('.jrn'):  # Filtrar solo archivos con extensión .jrn
                ruta_completa = os.path.join(ruta_subcarpeta, archivo)
                logs_df = pd.read_csv(ruta_completa, sep=';', encoding='utf-8', names=column_names,header=None)  # Ajusta el separador si es necesario
                logs_dataframes.append(logs_df)  
        df_Logscompleto = pd.concat(logs_dataframes, ignore_index=True)   # Concatenar todos los DataFrames en uno solo

        # Paso 1: Filtrar filas que contienen el texto "ATM. 'Posting Transaction Result"
        dfLogsATM = df_Logscompleto[
            df_Logscompleto.apply(lambda row: row.astype(str).str.contains("ATM.     'Posting Transaction Result ").any(), axis=1)
        ]

        # Usar expresiones regulares para extraer los valores
        dfLogsATM['seqNumber'] = dfLogsATM["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
        dfLogsATM['AuthNumber'] = dfLogsATM["FilteredData"].str.extract(r'"AuthNumber":"(\d+)"').astype(str)
        dfLogsATM['CardNumber'] = dfLogsATM["FilteredData"].str.extract(r'"CardNumber":"(\d+)"').astype(str)
        dfLogsATM['HostIP'] = dfLogsATM["FilteredData"].str.extract(r'"HostIP":"([^"]+)"').astype(str)
        dfLogsATM['Amount'] = dfLogsATM["FilteredData"].str.extract(r'"Amount":([0-9]+(?:\.[0-9]+)?)')
        dfLogsATM['DispensedTotal'] = dfLogsATM["FilteredData"].str.extract(r'"DispensedTotal":([0-9]+(?:\.[0-9]+)?)')
        dfLogsATM['TransactionType'] = dfLogsATM["FilteredData"].str.extract(r'"TransactionType":"([^"]+)"').astype(str)
        dfLogsATM['Status'] = dfLogsATM["FilteredData"].str.extract(r'"Status":"([^"]+)"').astype(str)
        dfLogsATM['JournalName'] = (
            dfLogsATM["FilteredData"]
            .str.extract(r'\d+\s+([^\s]+)\s+\'Posting Transaction Result')  # Extraer el texto relevante
            .squeeze()  # Convertir a una Serie si es un DataFrame con una sola columna
            .str.replace(r'\.', '', regex=True)  # Eliminar el punto
        ).astype(str)

        
        df_LogsUnified = pd.concat([df_LogsUnified,dfLogsATM], ignore_index=True)

    elif os.path.isdir(ruta_subcarpeta) and os.path.basename(ruta_subcarpeta).lower() == 'cashadvance':
        print(ruta_subcarpeta)
        for archivo in os.listdir(ruta_subcarpeta):
            if archivo.endswith('.jrn'):  # Filtrar solo archivos con extensión .jrn
                ruta_completa = os.path.join(ruta_subcarpeta, archivo)
                logs_df = pd.read_csv(ruta_completa, sep=';', encoding='utf-8', names=column_names,header=None)  # Ajusta el separador si es necesario
                logs_dataframes.append(logs_df)  
        df_Logscompleto = pd.concat(logs_dataframes, ignore_index=True)   # Concatenar todos los DataFrames en uno solo

        # Paso 1: Filtrar filas que contienen el texto "ATM. 'Posting Transaction Result"
        dfLogsCA = df_Logscompleto[
            df_Logscompleto.apply(lambda row: row.astype(str).str.contains("Cash Advance Transaction Info. 'Posting Transaction Result").any(), axis=1)
        ]
        
        # Usar expresiones regulares para extraer los valores
        dfLogsCA['seqNumber'] = dfLogsCA["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
        dfLogsCA['AuthNumber'] = dfLogsCA["FilteredData"].str.extract(r'"AuthNumber":"(\d+)"').astype(str)
        dfLogsCA['CardNumber'] = dfLogsCA["FilteredData"].str.extract(r'"CardNumber":"(\d+)"').astype(str)
        dfLogsCA['HostIP'] = dfLogsCA["FilteredData"].str.extract(r'"HostIP":"([^"]+)"').astype(str)
        dfLogsCA['Amount'] = dfLogsCA["FilteredData"].str.extract(r'"Amount":([0-9]+(?:\.[0-9]+)?)')
        dfLogsCA['DispensedTotal'] = dfLogsCA["FilteredData"].str.extract(r'"DispensedTotal":([0-9]+(?:\.[0-9]+)?)')
        dfLogsCA['TransactionType'] = dfLogsCA["FilteredData"].str.extract(r'"TransactionType":"([^"]+)"').astype(str)
        dfLogsCA['Status'] = dfLogsCA["FilteredData"].str.extract(r'"Status":"([^"]+)"').astype(str)
        dfLogsCA['JournalName'] = (
            dfLogsCA["FilteredData"]
            .str.extract(r'------------>\s*(.*?)\s*Transaction Info\.')  # Extraer el texto relevante
            .squeeze()  # Convertir a una Serie si es un DataFrame con una sola columna
            .str.replace(r'\.', '', regex=True)  # Eliminar el punto
        ).astype(str)
                
        df_LogsUnified = pd.concat([df_LogsUnified,dfLogsCA], ignore_index=True)

    elif os.path.isdir(ruta_subcarpeta) and os.path.basename(ruta_subcarpeta).lower() == 'ticketredemption':
        print(ruta_subcarpeta)
        for archivo in os.listdir(ruta_subcarpeta):
            if archivo.endswith('.jrn'):  # Filtrar solo archivos con extensión .jrn
                ruta_completa = os.path.join(ruta_subcarpeta, archivo)
                logs_df_tr = pd.read_csv(ruta_completa, sep=';', encoding='utf-8', names=column_names,header=None)  # Ajusta el separador si es necesario
                logs_dataframes_tr.append(logs_df_tr)  
        df_Logscompleto_tr = pd.concat(logs_dataframes_tr, ignore_index=True)   # Concatenar todos los DataFrames en uno solo

        # Paso 1: Filtrar filas que contienen el texto "ATM. 'Posting Transaction Result"
        dfLogsTR = df_Logscompleto_tr[
            df_Logscompleto_tr.apply(lambda row: row.astype(str).str.contains('Posting: {"type":"TicketRedemption"').any(), axis=1)
        ]
        
        # Usar expresiones regulares para extraer los valores
        dfLogsTR['seqNumber'] = dfLogsTR["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
        dfLogsTR['TicketData'] = dfLogsTR["FilteredData"].str.extract(r'"TicketData":"([^"]+)"').astype(str)
        dfLogsTR['Amount'] = dfLogsTR["FilteredData"].str.extract(r'"Amount":([0-9]+(?:\.[0-9]+)?)')
        dfLogsTR['DispensedTotal'] = dfLogsTR["FilteredData"].str.extract(r'"DispensedTotal":([0-9]+(?:\.[0-9]+)?)')
        #dfLogsTR['type'] = dfLogsTR["FilteredData"].str.extract(r'"TransactionType":"([^"]+)"').astype(str)
        #dfLogsTR['Type'] = dfLogsTR["FilteredData"].str.extract(r'"type":"([^"]+)"').astype(str)
        dfLogsTR['Type'] = dfLogsTR["FilteredData"].str.extract(r'"type"\s*:\s*"([^"]+)"', expand=False)
        dfLogsTR['BillDenom_01_1'] = dfLogsTR["FilteredData"].str.extract(r'"BillCount_01":([0-9]+(?:\.[0-9]+)?)')        
        dfLogsTR['BillDenom_02_5'] = dfLogsTR["FilteredData"].str.extract(r'"BillCount_02":([0-9]+(?:\.[0-9]+)?)')        
        dfLogsTR['BillDenom_03_10'] = dfLogsTR["FilteredData"].str.extract(r'"BillCount_03":([0-9]+(?:\.[0-9]+)?)')        
        dfLogsTR['BillDenom_04_20'] = dfLogsTR["FilteredData"].str.extract(r'"BillCount_04":([0-9]+(?:\.[0-9]+)?)')        
        dfLogsTR['BillDenom_05_50'] = dfLogsTR["FilteredData"].str.extract(r'"BillCount_05":([0-9]+(?:\.[0-9]+)?)')
        dfLogsTR['BillDenom_06_100'] = dfLogsTR["FilteredData"].str.extract(r'"BillCount_06":([0-9]+(?:\.[0-9]+)?)')
        dfLogsTR['Status'] = dfLogsTR["FilteredData"].str.extract(r'"Status":"([^"]+)"').astype(str)
        dfLogsTR['JournalName'] = os.path.basename(ruta_subcarpeta).lower()

    elif os.path.isdir(ruta_subcarpeta) and os.path.basename(ruta_subcarpeta).lower() == 'billbreaking':
        print(ruta_subcarpeta)
        for archivo in os.listdir(ruta_subcarpeta):
            if archivo.endswith('.jrn'):  # Filtrar solo archivos con extensión .jrn
                ruta_completa = os.path.join(ruta_subcarpeta, archivo)
                logs_df_bb = pd.read_csv(ruta_completa, sep=';', encoding='utf-8', names=column_names,header=None)  # Ajusta el separador si es necesario
                logs_dataframes_bb.append(logs_df_bb)  
        df_Logscompleto_bb = pd.concat(logs_dataframes_bb, ignore_index=True)   # Concatenar todos los DataFrames en uno solo

        # Paso 1: Filtrar filas que contienen el texto "ATM. 'Posting Transaction Result"
        dfLogsbb = df_Logscompleto_bb[
            df_Logscompleto_bb.apply(lambda row: row.astype(str).str.contains(' Posting: {"type":"BillBreak"').any(), axis=1)
        ]
        
        # Usar expresiones regulares para extraer los valores
        dfLogsbb['seqNumber'] = dfLogsbb["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
        dfLogsbb['TicketData'] = dfLogsbb["FilteredData"].str.extract(r'"TicketData":"([^"]+)"').astype(str)
        dfLogsbb['Amount'] = dfLogsbb["FilteredData"].str.extract(r'"Amount":([0-9]+(?:\.[0-9]+)?)')
        dfLogsbb['DispensedTotal'] = dfLogsbb["FilteredData"].str.extract(r'"DispensedTotal":([0-9]+(?:\.[0-9]+)?)')
        #dfLogsTR['type'] = dfLogsTR["FilteredData"].str.extract(r'"TransactionType":"([^"]+)"').astype(str)
        #dfLogsTR['Type'] = dfLogsTR["FilteredData"].str.extract(r'"type":"([^"]+)"').astype(str)
        dfLogsbb['Type'] = dfLogsbb["FilteredData"].str.extract(r'"type"\s*:\s*"([^"]+)"', expand=False)        
        dfLogsbb['BillDenom_01_1'] = dfLogsbb["FilteredData"].str.extract(r'"BillCount_01":([0-9]+(?:\.[0-9]+)?)')        
        dfLogsbb['BillDenom_02_5'] = dfLogsbb["FilteredData"].str.extract(r'"BillCount_02":([0-9]+(?:\.[0-9]+)?)')        
        dfLogsbb['BillDenom_03_10'] = dfLogsbb["FilteredData"].str.extract(r'"BillCount_03":([0-9]+(?:\.[0-9]+)?)')        
        dfLogsbb['BillDenom_04_20'] = dfLogsbb["FilteredData"].str.extract(r'"BillCount_04":([0-9]+(?:\.[0-9]+)?)')        
        dfLogsbb['BillDenom_05_50'] = dfLogsbb["FilteredData"].str.extract(r'"BillCount_05":([0-9]+(?:\.[0-9]+)?)')
        dfLogsbb['BillDenom_06_100'] = dfLogsbb["FilteredData"].str.extract(r'"BillCount_06":([0-9]+(?:\.[0-9]+)?)')
        dfLogsbb['Status'] = dfLogsbb["FilteredData"].str.extract(r'"Status":"([^"]+)"').astype(str)
        dfLogsbb['JournalName'] = os.path.basename(ruta_subcarpeta).lower()

print(dfLogsbb)

#df_LogsUnified.to_excel(output_path, index=False, engine='openpyxl')  # Usa openpyxl como motor
#print(f"Archivo exportado exitosamente a {output_path}")

# Guardar el primer dataframe en la primera hoja
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df_LogsUnified.to_excel(writer, sheet_name='CARDLOGS', index=False)

# Agregar el segundo dataframe en la segunda hoja
with pd.ExcelWriter(output_path,mode='a',engine='openpyxl') as writer:
    dfLogsTR.to_excel(writer, sheet_name='TRLOGS', index=False)

with pd.ExcelWriter(output_path,mode='a',engine='openpyxl') as writer:
    dfLogsbb.to_excel(writer, sheet_name='BBLOGS', index=False)

C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\Logs\ATM


C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsATM['seqNumber'] = dfLogsATM["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsATM['AuthNumber'] = dfLogsATM["FilteredData"].str.extract(r'"AuthNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:33: SettingWithCo

C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\Logs\BILLBREAKING


C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsbb['seqNumber'] = dfLogsbb["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsbb['TicketData'] = dfLogsbb["FilteredData"].str.extract(r'"TicketData":"([^"]+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:129: SettingWithC

C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\Logs\CASHADVANCE


C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsCA['seqNumber'] = dfLogsCA["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsCA['AuthNumber'] = dfLogsCA["FilteredData"].str.extract(r'"AuthNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:66: SettingWithCopyWa

C:\Users\paula\UNIR\MasterBigDatayVisualAnalytics\cuatrimestre2\TFM-TFE\Entrega3\TESIS_Maestria_ETL\ETLProyect\ArchivosEntrada\Logs\TICKETREDEMPTION


C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsTR['seqNumber'] = dfLogsTR["FilteredData"].str.extract(r'"seqNumber":"(\d+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLogsTR['TicketData'] = dfLogsTR["FilteredData"].str.extract(r'"TicketData":"([^"]+)"').astype(str)
C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3243722929.py:98: SettingWithCopy

                                             FilteredData         seqNumber  \
404     00:00:31 402001 ' Posting: {"type":"BillBreak"...  0823202400000976   
809     00:01:07 402001 ' Posting: {"type":"BillBreak"...  0823202400004898   
1214    00:16:35 402001 ' Posting: {"type":"BillBreak"...  0823202400162040   
1619    00:40:11 402001 ' Posting: {"type":"BillBreak"...  0823202400395160   
2024    00:52:20 402001 ' Posting: {"type":"BillBreak"...  0823202400515980   
...                                                   ...               ...   
810968  23:46:10 402001 ' Posting: {"type":"BillBreak"...  0831202423454842   
811373  23:48:12 402001 ' Posting: {"type":"BillBreak"...  0831202423475349   
811778  23:48:41 402001 ' Posting: {"type":"BillBreak"...  0831202423482207   
812183  23:51:15 402001 ' Posting: {"type":"BillBreak"...  0831202423504148   
812588  23:51:58 402001 ' Posting: {"type":"BillBreak"...  0831202423512574   

              TicketData Amount DispensedTotal     

In [29]:
#--------------------------------------------------------------------------LiveOffice BB and TR Reports----------------------------------------------------------------------------------------------

# Obtener una lista de todos los archivos Excel en la carpeta
file_list = [os.path.join(folder_path_tr_bb, f) for f in os.listdir(folder_path_tr_bb) if f.endswith(('.xls', '.xlsx'))]

dataframes = []
for file in file_list:
    file_name = os.path.basename(file)
    if "Voucher_Redemption_Transaction" in file_name:
        print("El archivo contiene 'Voucher Redemption' en el nombre.")
        df_tr = pd.read_excel(file, skiprows=15, header=None)
        new_columns = df_tr.iloc[0].astype(str) + df_tr.iloc[1].astype(str)
        df_tr.columns = new_columns  # Asignar los nuevos encabezados
        df_tr = df_tr[2:]  # Eliminar las filas de encabezado original (0, 1 y 2)
        df_tr.reset_index(drop=True, inplace=True)
        #print (df.head())
        df_tr['file_name'] = os.path.basename(file)# Agregar el nombre del archivo    
        print("Nombre del archivo:", file_name)
        #Tranformación 1 - Nulos
        # Eliminar columnas donde todos los valores son NaN
        print ("Transformacion Nulos\n")
        df_tr = df_tr.dropna(axis=1, how='all')
        #Tranformación 3 - Reemplaza NaN con un valor predeterminado si es necesario
        #df = df.fillna(0)    
        # Encuentra el nombre exacto de la columna que contiene 'DATE & TIME'
        target_column = df_tr.filter(like="DATE & TIME").columns[0]
        # Renombra dinámicamente la columna encontrada
        df_tr = df_tr.rename(columns={target_column: "DATE & TIME"})
        df_tr = df_tr.loc[df_tr["DATE & TIME"] != "Totals:"]
        #print(df) 
        columnas_a_convertir = ["TRANSAMOUNT($)", "DISPENSED QTYS$1", "nan$5", "nan$10", "nan$20", "nan$50", "nan$100"]
        df_tr[columnas_a_convertir] = df_tr[columnas_a_convertir].astype(float)
        # Rellenar valores NaN con un número válido (ejemplo: 0)
        df_tr.fillna(0, inplace=True)
    elif "Bill_Breaking_Transaction" in file_name:
        print("El archivo contiene 'Bill Breaking' en el nombre.")        
        df_bb = pd.read_excel(file, sheet_name="Sheet2", skiprows=3, header=None)
        file_name = os.path.basename(file)
        new_columns = df_bb.iloc[2].astype(str) + df_bb.iloc[3].astype(str)
        df_bb.columns = new_columns  # Asignar los nuevos encabezados
        df_bb = df_bb[4:]  # Eliminar las filas de encabezado original (0, 1 y 2)
        df_bb.reset_index(drop=True, inplace=True)
        #print (df.head())
        df_bb['file_name'] = os.path.basename(file)# Agregar el nombre del archivo    
        print("Nombre del archivo:", file_name)
        #Tranformación 1 - Nulos
        # Eliminar columnas donde todos los valores son NaN
        print ("Transformacion Nulos\n")
        df_bb = df_bb.dropna(axis=1, how='all')
        #Tranformación 3 - Reemplaza NaN con un valor predeterminado si es necesario
        #df = df.fillna(0)    
        # Encuentra el nombre exacto de la columna que contiene 'DATE & TIME'
        target_column = df_bb.filter(like="DATE & TIME").columns[0]
        # Renombra dinámicamente la columna encontrada
        df_bb = df_bb.rename(columns={target_column: "DATE & TIME"})
        df_bb = df_bb.loc[df_bb["DATE & TIME"] != "Totals:"]
        #print(df) 
    
#df_bb.to_excel(output_path3, index=False, engine='openpyxl')  # Usa openpyxl como motor
#print(f"Archivo exportado exitosamente a {output_path3}")

# Guardar el primer dataframe en la primera hoja
with pd.ExcelWriter(output_path3, engine='openpyxl') as writer:
    df_tr.to_excel(writer, sheet_name='TRLO', index=False)

# Agregar el segundo dataframe en la segunda hoja
with pd.ExcelWriter(output_path3,mode='a',engine='openpyxl') as writer:
    df_bb.to_excel(writer, sheet_name='BBLO', index=False)

El archivo contiene 'Bill Breaking' en el nombre.
Nombre del archivo: Bill_Breaking_Transaction-20240903-0921.xls
Transformacion Nulos

El archivo contiene 'Voucher Redemption' en el nombre.
Nombre del archivo: Voucher_Redemption_Transaction-20240903-0928.xls
Transformacion Nulos



C:\Users\paula\AppData\Local\Temp\ipykernel_120716\3966079688.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_tr.fillna(0, inplace=True)


In [13]:

print(type(dfLogsTR))

<class 'pyspark.sql.dataframe.DataFrame'>


In [40]:
#------------------------------------------------------------------SPARK Consolidado logs

spark = SparkSession.builder.appName("Logs_LO TR_BB Comparative").getOrCreate()

#Pandas a Spark
#print(dfLogsTR.head())
#dfLogsTR = dfLogsTR.toPandas()
dfLOGSTR = spark.createDataFrame(dfLogsTR)
dfLogsBB = spark.createDataFrame(dfLogsbb)


#print (df_tr)
dfLOTR = spark.createDataFrame(df_tr)
dfLOBB = spark.createDataFrame(df_bb) 


#Renombre de columnas
#dfLO = dfLO.withColumnRenamed("HOSTSEQ NUMBER", "hostseq_number")
#dfHOST = dfHOST.withColumnRenamed("Seq", "seq")

# Filtrar el DataFrame dfLO
#dfLO = dfLO.filter(F.col("STATUS") != "Denied")
#dfLO = dfLO.filter(F.col("TRANSACTIONTYPE") != "Balance inquiry")


#Verificación
print('Dataframe de LiveOffice')
dfLOBB.printSchema()
print('Dataframe de Logs')
dfLogsBB.printSchema()

#dfLOGSTR = dfLOGSTR.filter(F.col("SEQUENCENUMBER") != "")


#-Join Logs con LOHOSTUNIFIED
joined_df1 = dfLOGSTR.join(
    dfLOTR,
    dfLOGSTR["seqNumber"] == dfLOTR["SEQUENCENUMBER"],
    "left"
)

sorted_df1 = joined_df1.orderBy("seqNumber")


# Agregar columna de indicador

result_df2 = sorted_df1.withColumn(
    "found_in_Systems",
    F.when(F.col("SEQUENCENUMBER").isNotNull(), True).otherwise(False)
)

result_df2.show()

#-Join Logs con HOST
joined_df2 = dfLogsBB.join(
    dfLOBB,
    dfLogsBB["seqNumber"] == dfLOBB["SEQUENCENUMBER"],
    "left"
)

sorted_df2 = joined_df2.orderBy("seqNumber")

# Agregar columna de indicador
result_df3 = sorted_df2.withColumn(
    "found_in_datastream",
    F.when(F.col("SEQUENCENUMBER").isNotNull(), True).otherwise(False)
)

result_df3.show()

pandas_df1 = result_df2.toPandas()
pandas_df2 = result_df3.toPandas()

with pd.ExcelWriter(output_path3, engine='openpyxl') as writer:
    pandas_df1.to_excel(writer, sheet_name='LogsTR_LO', index=False)  # Guardar df1 en 'Hoja1'
    pandas_df2.to_excel(writer, sheet_name='LogsBB_LO', index=False)  # Guardar df2 en 'Hoja2'



#pandas_df.to_excel(output_path3, index=False, engine='openpyxl')  # Usa openpyxl como motor
print(f"Archivo exportado exitosamente a {output_path3}")

# Detener sesión de Spark
#spark.stop()

Dataframe de LiveOffice
root
 |-- DATE & TIME: string (nullable = true)
 |-- SEQUENCENUMBER: string (nullable = true)
 |-- TRANSAMOUNT($): long (nullable = true)
 |-- DISPENSED QTYS$1: long (nullable = true)
 |-- nan$5: long (nullable = true)
 |-- nan$10: long (nullable = true)
 |-- nan$20: long (nullable = true)
 |-- nan$50: long (nullable = true)
 |-- nan$100: long (nullable = true)
 |-- DISPENSED($): long (nullable = true)
 |-- MANUALLYPAID ($): long (nullable = true)
 |-- nanSTATUS: string (nullable = true)
 |-- file_name: string (nullable = true)

Dataframe de Logs
root
 |-- FilteredData: string (nullable = true)
 |-- seqNumber: string (nullable = true)
 |-- TicketData: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- DispensedTotal: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- JournalName: string (nullable = true)

+--------------------+----------------+--------------------+------+--------------+-------